In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
import numpy as np
import torch.nn.functional as F
from einops import rearrange
import glob
from pdf2image import convert_from_path
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.utils import ImageReader
import math

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR'

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, _attn_implementation='flash_attention_2', 
                                 trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# -------------------------- 核心配置（用户可调整） --------------------------
INPUT_PDF = "../MMLongBench-Doc/data/documents/"               # 输入PDF路径
OUTPUT_PDF = "pdf_output/"         # 输出拼接PDF路径
IMAGE_INPUT = "image_input/"
IMAGE_OUTPUT = "image_output/"    # OCR结果保存目录
IMAGE_TEMP_PATH = "temp/"
TARGET_TOKENS_LENGTH = 256        # 目标Vision Tokens长度（H_patch × W_patch）
PATCH_SIZE = 16                   # DeepSeek-OCR默认Patch Size（固定为16）
BASE_SIZE = 1024                  # 模型基础输入尺寸
IMAGE_SIZE = 640                  # 模型实际输入尺寸（控制resize上限）Gundam
CROP_MODE = True                  # 模型裁剪模式（与原推理逻辑一致）
SAVE_OCR_RESULTS = True           # 是否保存OCR文本结果
# WHITE_DOC_LIST = ['05-03-18-political-release']
# ratio = "text_tokens" / (1024/16)*(1024/16)/16

In [3]:
def calculate_resize_padding_for_1024x1024(original_size, patch_size=16):
    """
    保持长宽比不变，计算图像Resize+Padding的参数，最终分辨率固定为1024×1024
    Args:
        original_size: (width, height) 原图像尺寸（PIL.Image.size返回格式）
        patch_size: 模型Patch Size（DeepSeek-OCR默认16，无需修改）
    Returns:
        resize_size: (new_width, new_height) Resize后的尺寸（保持长宽比，Patch Size整数倍）
        padding: (pad_left, pad_top, pad_right, pad_bottom) 上下左右补白量（最终凑11024×1024）
        actual_tokens: 实际Vision Tokens长度（(1024/patch_size) × (10240/patch_size) = 4096）
    """
    target_resolution = (1024, 1024)  # 固定目标分辨率：1280×1280
    original_w, original_h = original_size
    aspect_ratio = original_w / original_h  # 原始宽高比（保持不变）

    # 第一步：计算Resize尺寸（保持长宽比，不超过1280×1280，且为Patch Size整数倍）
    scale = min(
        target_resolution[0] / original_w,
        target_resolution[1] / original_h
    )
    resize_w = int(original_w * scale)
    resize_h = int(original_h * scale)
    # 调整为Patch Size的整数倍（确保模型编码时Tokens无残留）
    resize_w = (resize_w // patch_size) * patch_size
    resize_h = (resize_h // patch_size) * patch_size
    # 边界修正：避免极端小图像导致尺寸为0
    resize_w = max(resize_w, patch_size)
    resize_h = max(resize_h, patch_size)

    # 第二步：计算Padding参数（上下左右补白，图像居中，最终凑1280×1280）
    pad_total_w = target_resolution[0] - resize_w  # 横向总补白量
    pad_total_h = target_resolution[1] - resize_h  # 纵向总补白量
    # 对称补白（图像居中显示）
    pad_left = pad_total_w // 2
    pad_right = pad_total_w - pad_left
    pad_top = pad_total_h // 2
    pad_bottom = pad_total_h - pad_top

    # 第三步：计算固定Vision Tokens数量（1280/Patch Size 为固定值）
    patch_h = target_resolution[1] // patch_size
    patch_w = target_resolution[0] // patch_size
    actual_tokens = patch_h * patch_w

    return (resize_w, resize_h), (pad_left, pad_top, pad_right, pad_bottom), actual_tokens

In [4]:
def process_all_pdf(PDF_INPUT):
    pdf_dir = PDF_INPUT
    if not os.path.exists(pdf_dir) or not os.path.isdir(pdf_dir):
        print(f"错误：目录不存在或不是文件夹 → {pdf_dir}")
    else:
        # 遍历目录，筛选PDF文件
        pdf_files_info = []  # 存储 (无扩展名文件名, 完整路径) 列表
        for file_name in os.listdir(pdf_dir):
            # 拼接文件完整路径
            file_path = os.path.join(pdf_dir, file_name)
            # 筛选：仅保留文件（排除子目录）+ 后缀为 .pdf（忽略大小写）
            if os.path.isfile(file_path) and file_name.lower().endswith(".pdf"):
                # 提取无扩展名文件名（核心：分割文件名和扩展名）
                file_name_without_ext = os.path.splitext(file_name)[0]
                pdf_files_info.append( (file_name_without_ext, file_path) )
                new_path = os.path.join(IMAGE_TEMP_PATH, file_name_without_ext)
                os.makedirs(new_path, exist_ok=True)
            process_pdf_ocr(file_path, new_path)
        
        if not pdf_files_info:
            print(f"提示：目录 {pdf_dir} 下未找到PDF文件")
        else:
            print(f"成功找到 {len(pdf_files_info)} 个PDF文件：")
            print("-" * 50)
            for name, path in pdf_files_info:
                print(f"文件名：{name}")
                print(f"完整路径：{path}")
                print("-" * 30)

In [5]:
def process_pdf_ocr(pdf_file, output_path):
    # 1. PDF转图像
    pdf_images = pdf_to_images(pdf_file)
    original_sizes = [img.size for img in pdf_images]  # 保存原尺寸（width, height）
    processed_images = []  # 存储处理后（OCR推理+还原尺寸）的图像

    # 2. 逐页resize
    for page_idx, (img, original_size) in enumerate(zip(pdf_images, original_sizes)):
        print(f"\n处理第{page_idx+1}/{len(pdf_images)}页...")

        # 2.1 计算Resize+Padding参数（固定目标1024×1024，保持长宽比）
        resize_size, padding, actual_tokens = calculate_resize_padding_for_1024x1024(
            original_size=original_size,
            patch_size=16  # DeepSeek-OCR默认Patch Size=16，无需修改
        )
        print(f"原尺寸：{original_size} → Resize尺寸：{resize_size} → Padding：{padding} → 最终尺寸：1024×1024 → 实际Tokens：{actual_tokens}")
        
        # 2.2 Resize图像（保持宽高比，LANCZOS插值保证清晰度）
        resized_img = img.resize(resize_size, Image.LANCZOS)
        
        # 2.3 Padding至1024×1024（白色背景，图像居中）
        target_img = Image.new("RGB", (1024, 1024), color="white")  # 创建目标尺寸画布
        pad_left, pad_top, _, _ = padding
        target_img.paste(resized_img, (pad_left, pad_top))  # 粘贴图像到居中位置
    
        # 后续可使用 target_img 进行模型推理（已固定为1024×1024，保持原长宽比）
        path = os.path.join(output_path,"temp_page_" + str(page_idx+1) + ".jpg")
        resized_img.save(path, quality=95)
        print(f"处理后的图像已保存，尺寸：{target_img.size}")

In [6]:
def pdf_to_images(pdf_path, dpi=300):
    """将PDF每页转为PIL图像（dpi控制原始分辨率）"""
    try:
        images = convert_from_path(
            pdf_path,
            dpi=dpi,
            fmt="jpg",
            thread_count=4,
            use_cropbox=True,
            strict=False
        )
        print(f"成功读取PDF：{pdf_path}，共{len(images)}页")
        return images
    except Exception as e:
        print(f"PDF转图像失败：{e}")
        raise

In [7]:
def read_ocr_result_from_file(model_output_dir):
    """从模型输出目录读取OCR结果（适配DeepSeek-OCR格式）"""
    # 查找模型生成的结果文件（优先txt/md）
    result_files = []
    for ext in ['*.txt', '*.md']:
        result_files.extend(glob.glob(os.path.join(model_output_dir, ext)))
    
    if not result_files:
        print(f"警告：{model_output_dir} 中未找到OCR结果文件")
        return ""
    
    # 读取第一个结果文件（模型默认只生成一个）
    with open(result_files[0], "r", encoding="utf-8", errors="ignore") as f:
        ocr_text = f.read().strip()
    return ocr_text

In [12]:
from pathlib import Path
def convert_all_pdf(all_pdf_images_dir, output_dir):
    all_pdf_images_dir = Path(all_pdf_images_dir)
    output_dir = Path(output_dir)
    # 1. 确保主输入目录存在
    all_pdf_images_dir.mkdir(exist_ok=True)

    # 2. 创建主输出目录（不存在则创建，已存在则跳过）
    output_dir.mkdir(exist_ok=True)
    print(f"✅ 主输出目录已准备：{output_dir.resolve()}")
    print("-" * 50)

    # 3. 遍历主输入目录下的所有子目录（images_input_subdir）
    images_input_subdirs = [subdir for subdir in all_pdf_images_dir.iterdir() if subdir.is_dir()]
    
    if not images_input_subdirs:
        print(f"⚠️  警告：主输入目录 {all_pdf_images_dir.name} 下无任何子目录")
        return

    # 4. 逐个处理子目录
    for idx, images_input_subdir in enumerate(images_input_subdirs, 1):
        subdir_name = images_input_subdir.name  # 子目录名称（如"subdir_1"）
        print(f"\n📌 处理第{idx}/{len(images_input_subdirs)}个输入子目录：{subdir_name}")
        # if subdir_name not in WHITE_DOC_LIST:
        #     print(f"{subdir_name} not in white_doc_list")
        #     continue

        # 5. 创建对应的输出子目录（images_output_subdir）
        # 输出子目录路径：output_dir / 输入子目录名（与输入子目录同名，避免冲突）
        images_output_subdir = output_dir / subdir_name
        images_output_subdir.mkdir(exist_ok=True)  # 不存在则创建
        print(f"  ✅ 输出子目录已创建：{images_output_subdir.resolve()}")

        # 6. 遍历当前输入子目录下的所有jpg文件（支持大小写）
        jpg_files = list(images_input_subdir.glob(f"*"))  # 先获取所有文件
        jpg_files = [f for f in jpg_files if f.is_file()]  # 筛选jpg文件
        page_num = len(jpg_files)

        if not jpg_files:
            print(f"  ⚠️  警告：该子目录下无任何jpg文件，跳过函数调用")
            print("-" * 30)
            continue

        # 7. 调用转换函数（传入输入子目录和输出子目录）
        try:
            convert_images_to_markdown(images_input_subdir, images_output_subdir, page_num)
            print(f"  ✅ 函数调用成功：convert_images_to_markdown({images_input_subdir}, {images_output_subdir})")
        except Exception as e:
            print(f"  ❌ 函数调用失败：{str(e)}")
        print("-" * 30)

        # 8. 调用mmd2pdf转换函数，将markdown文件转为pdf
        try:
            pdf_output = os.path.join(OUTPUT_PDF, subdir_name)
            mmd2pdf(pdf_output, page_num)
            print(f"  ✅ 函数调用成功：mmd2pdf({pdf_output}, {page_num})")
        except Exception as e:
            print(f"  ❌ 函数调用失败：{str(e)}")
        print("-" * 30)

    print("\n🎉 所有子目录处理完毕！")
    print(f"📊 统计：共处理 {len(images_input_subdirs)} 个输入子目录")
    print(f"📁 输出文件存放位置：{output_dir.resolve()}")

In [13]:
def convert_images_to_markdown(images_input_path, output_dir, page_num):
    print("images_input_path:",images_input_path)
    print("output_dir:", output_dir)
    print("page_num:", page_num)
    for idx in range(page_num):
        img_path = os.path.join(images_input_path, "temp_page_" + str(idx+1) + ".jpg")
        output_file_path = os.path.join(output_dir,"page_"+str(idx+1))
        os.makedirs(output_file_path, exist_ok=True)
        convert_img_to_markdown(img_path, output_file_path)
        print(idx)
    

In [14]:
def convert_img_to_markdown(image_file, output_dir):
    # -------------------------- 执行OCR生成Markdown --------------------------
    prompt = "<image>\n<|grounding|>Convert the document to markdown. "
    # 调用模型infer生成Markdown（复用原逻辑）
    res = model.infer(
        tokenizer=tokenizer,
        prompt=prompt,
        image_file=image_file,
        output_path=output_dir,
        base_size=BASE_SIZE,
        image_size=IMAGE_SIZE,
        crop_mode=CROP_MODE,
        save_results=True,  # 保存Markdown文件到output_dir
        test_compress=True
    )
    # 提取生成的Markdown文本（根据模型返回格式调整，通常为字符串或字典的text字段）
    print("✅ markdown生成完成！")

In [15]:
import os
import shutil
import re
import markdown2
import imgkit
from fpdf import FPDF
from PIL import Image
import subprocess

# -------------------------- Ubuntu环境专属配置 --------------------------
ROOT_DIR = "image_output"  # 你的mmd文件总目录（Ubuntu下路径无中文更稳定）
IMG_OUTPUT_DIR = "mmd_to_images"  # 单页图像保存目录
IMAGE_WIDTH = 1920  # 图像宽度（px），0.12.6支持该参数
IMAGE_QUALITY = 95  # 图像质量（1-100）
WKHTMLTOIMAGE_PATH = "wkhtmltoimage"  # Ubuntu下无需指定（已在PATH中）

# -------------------------- 修复root用户环境变量（关键！） --------------------------
def fix_root_env():
    """解决root用户XDG_RUNTIME_DIR未设置的警告，避免潜在环境错误"""
    if os.geteuid() == 0:  # 检测是否为root用户
        os.environ.setdefault("XDG_RUNTIME_DIR", "/tmp/runtime-root")
        os.makedirs(os.environ["XDG_RUNTIME_DIR"], exist_ok=True)
        os.chmod(os.environ["XDG_RUNTIME_DIR"], 0o700)  # 设置权限
        print(f"⚠️  已为root用户设置XDG_RUNTIME_DIR：{os.environ['XDG_RUNTIME_DIR']}")

# -------------------------- 步骤1：初始化目录 --------------------------
def init_dirs():
    if os.path.exists(IMG_OUTPUT_DIR):
        shutil.rmtree(IMG_OUTPUT_DIR)
    os.makedirs(IMG_OUTPUT_DIR)
    print(f"✅ 初始化图像输出目录：{IMG_OUTPUT_DIR}")

# -------------------------- 步骤2：路径修复（显式添加file://协议） --------------------------
def fix_and_check_image_paths(mmd_content, images_dir):
    """
    关键修复：Ubuntu下必须显式指定file://协议，格式为 file:///绝对路径（三个斜杠）
    例：/home/root/image_output/page_1/images/0.jpg → file:///home/root/image_output/page_1/images/0.jpg
    """
    image_pattern = r"!\[(.*?)\]\((images/[^)]+)\)"
    missing_images = []

    def replace_and_check(match):
        desc = match.group(1)
        rel_path = match.group(2)
        img_filename = os.path.basename(rel_path)
        abs_path = os.path.abspath(os.path.join(images_dir, img_filename))
        
        # 验证图片存在性
        if not os.path.exists(abs_path) or os.path.getsize(abs_path) < 1024:
            missing_images.append(abs_path)
            return f"![{desc}（图片缺失）]({abs_path})"
        
        # 显式添加file://协议（Ubuntu 0.12.6必需）
        file_url = f"file://{abs_path}"  # 注意：Ubuntu下是file://+绝对路径（自动转为三个斜杠）
        return f"![{desc}]({file_url})"

    fixed_mmd = re.sub(image_pattern, replace_and_check, mmd_content)
    if missing_images:
        print(f"⚠️  缺失图片：{missing_images}")
    return fixed_mmd
# -------------------------- 步骤3：mmd转图像（核心修复参数） --------------------------
def mmd_to_image(page_dir, page_num):
    mmd_path = os.path.join(page_dir, "result.mmd")
    images_dir = os.path.join(page_dir, "images")
    
    if not os.path.exists(mmd_path):
        print(f"❌ 第{page_num}页：未找到result.mmd，跳过")
        return None
    if not os.path.exists(images_dir):
        print(f"❌ 第{page_num}页：未找到images文件夹，跳过")
        return None
    
    # 1. 读取并修复路径（添加file://）
    with open(mmd_path, "r", encoding="utf-8") as f:
        mmd_content = f.read()
    fixed_mmd = fix_and_check_image_paths(mmd_content, images_dir)
    
    # 2. 转换为HTML
    html_content = markdown2.markdown(
        fixed_mmd,
        extras=["fenced-code-blocks", "tables", "footnotes"]
    )
    
    # 3. 构建HTML（简化，避免解析问题）
    # full_html = f"""
    # <!DOCTYPE html>
    # <html>
    # <head>
    #     <meta charset="UTF-8">
    #     <style>
    #         body {{ padding: 10px; max-width: {IMAGE_WIDTH}px; margin: 0; }}
    #         img {{ max-width: 100%; height: auto; }}
    #     </style>
    # </head>
    # <body>{html_content}</body>
    # </html>
    # """
    full_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            /* 核心修改：让HTML内容铺满PDF页面，清除所有默认边距和宽度限制 */
            html, body {{
                margin: 0 !important;    /* 清除页面默认外边距（关键） */
                padding: 0 !important;   /* 清除页面默认内边距（原代码是10px，导致留白） */
                width: 100% !important;  /* 宽度自适应PDF页面 */
                max-width: 100% !important;  /* 取消固定max-width限制（原代码是IMAGE_WIDTH，导致无法铺满） */
                box-sizing: border-box;  /* 避免padding/border影响总宽度 */
                background-color: white; /* 确保背景为白色，适配PDF打印 */
            }}
            /* 保留图片自适应：避免图片超出页面宽度 */
            img {{
                max-width: 100% !important;
                height: auto !important;
                display: block;  /* 消除图片下方多余空白 */
            }}
            /* 可选：优化文本排版（如需保留一定内边距，可调整这里） */
            /* 若需要内容与页面边缘有少量距离，可改为 padding: 5px !important; */
            .content-container {{
                padding: 0 !important;  /* 完全铺满时设为0，需要留白时调整数值 */
            }}
            /* 兼容markdown2转换后的表格/代码块样式，避免溢出 */
            table {{
                width: 100% !important;
                border-collapse: collapse;
            }}
            pre {{
                white-space: pre-wrap;  /* 代码块自动换行，避免超出页面 */
                word-break: break-all;
            }}
        </style>
    </head>
    <!-- 包裹一层容器，方便后续调整内边距（可选） -->
    <body>
        <div class="content-container">{html_content}</div>
    </body>
    </html>
    """
    
    # 4. 保存临时HTML
    temp_html_path = os.path.join(page_dir, f"temp_page_{page_num}.html")
    with open(temp_html_path, "w", encoding="utf-8") as f:
        f.write(full_html)
    
    # 5. 关键参数（添加--enable-local-file-access，Ubuntu 0.12.6支持）
    img_output_path = os.path.join(IMG_OUTPUT_DIR, f"page_{page_num}.png")
    imgkit_options = {
        "format": "png",
        "width": IMAGE_WIDTH,
        "quality": IMAGE_QUALITY,
        "encoding": "UTF-8",
        "quiet": "",  # 隐藏冗余输出
        "enable-local-file-access": "",  # 允许本地文件访问（解决协议错误的核心！）
    }
    
    # 配置
    config = imgkit.config(wkhtmltoimage=WKHTMLTOIMAGE_PATH)
    
    # 执行转换（捕获详细错误）
    try:
        # 直接调用subprocess执行命令（绕过imgkit可能的参数传递问题）
        cmd = [
            WKHTMLTOIMAGE_PATH,
            "--format", "png",
            "--width", str(IMAGE_WIDTH),
            "--quality", str(IMAGE_QUALITY),
            "--encoding", "UTF-8",
            "--quiet",
            "--enable-local-file-access",  # 显式启用本地文件访问
            temp_html_path,
            img_output_path
        ]

        print(WKHTMLTOIMAGE_PATH)
        print(temp_html_path)
        print(img_output_path)
        # 执行命令并捕获输出
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            env=os.environ  # 传递修复后的环境变量
        )
        
        # 验证转换结果
        if os.path.exists(img_output_path) and os.path.getsize(img_output_path) > 2048:
            print(f"✅ 第{page_num}页：转换成功 → {os.path.basename(img_output_path)}")
            return img_output_path
        else:
            print(f"❌ 第{page_num}页：转换失败")
            print(f"  命令输出：{result.stdout}")
            print(f"  错误信息：{result.stderr}")
            return None
    except Exception as e:
        print(f"❌ 第{page_num}页：报错 → {str(e)}")
        return None
    finally:
        # 强制删除临时文件
        if os.path.exists(temp_html_path):
            os.remove(temp_html_path)

# -------------------------- 步骤4：合并图像为PDF（保持稳定） --------------------------
def images_to_pdf(image_paths, pdf_output):
    valid_images = [
        img for img in image_paths 
        if img and os.path.exists(img) and os.path.getsize(img) > 2048
    ]
    
    if not valid_images:
        print("\n❌ 无有效图像可合并，PDF生成失败")
        return
    
    valid_images.sort(key=lambda x: int(os.path.basename(x).split("_")[1].split(".")[0]))
    print(f"\n📊 共{len(valid_images)}页有效图像，合并PDF中...")
    
    pdf = FPDF(unit="mm", format="A4")
    pdf.set_auto_page_break(auto=True, margin=10)
    
    for img_path in valid_images:
        try:
            with Image.open(img_path) as img:
                img_w_px, img_h_px = img.size
            
            img_w_mm = (img_w_px / 96) * 25.4
            img_h_mm = (img_h_px / 96) * 25.4
            
            max_w = 210 - 20
            max_h = 297 - 20
            scale = min(max_w / img_w_mm, max_h / img_h_mm, 1.0)
            
            pdf.add_page()
            pdf.image(
                img_path,
                x=(210 - img_w_mm * scale) / 2,
                y=10,
                w=img_w_mm * scale,
                h=img_h_mm * scale
            )
        except Exception as e:
            print(f"⚠️  跳过异常图像：{os.path.basename(img_path)} → {str(e)}")

    os.makedirs(os.path.join(OUTPUT_PDF, ), exist_ok=True)
    pdf.output(pdf_output)
    print(f"\n🎉 PDF合并完成！路径：{os.path.abspath(pdf_output)}")

def mmd2pdf(pdf_output_path, page_num):   
    # pdf_output_path 需要是文件名
    image_paths = []
    for page_idx in range(1, page_num):
        page_dir = os.path.join(pdf_output_path,f"page_{page_idx}")
        print(f"\n--- 处理第{page_num}页（{page_dir}）---")
        img_path = mmd_to_image(page_dir, page_idx)
        image_paths.append(img_path)
    
    # 合并PDF
    print("\n" + "="*50)
    images_to_pdf(image_paths, pdf_output_path)
    
    # 输出总结
    print("\n" + "="*50)
    print("📊 流程总结：")
    print(f" - 总页面数：16")
    print(f" - 成功转换：{len([x for x in image_paths if x])}页")
    print(f" - 图像目录：{os.path.abspath(IMG_OUTPUT_DIR)}")
    print(f" - 最终PDF：{os.path.abspath(pdf_output_path)}")
    print("="*50)

In [16]:
if __name__ == "__main__":

    # 修复root用户环境变量（关键步骤）
    fix_root_env()
    # 初始化目录
    init_dirs()
    
    # 创建输出目录（如果不存在）
    os.makedirs(IMAGE_OUTPUT, exist_ok=True)
    os.makedirs(IMAGE_TEMP_PATH, exist_ok=True)
    
    # 执行主流程
    # process_all_pdf(INPUT_PDF)  # 把MMLongBench-Doc的所有pdf文档逐页resize到temp目录的同名子目录下，以1024*1024分辨率图像存放
    convert_all_pdf(IMAGE_TEMP_PATH,IMAGE_OUTPUT)  # 把temp/目录下所有子目录的图像，通过DeepSeek-ocr转为markdown文件

    # process_pdf_ocr('../MMLongBench-Doc/data/documents/05-03-18-political-release.pdf','temp/')
    # convert_images_to_markdown('temp/05-03-18-political-release','mmd_to_images/05-03-18-political-release',17)
    # mmd2pdf('mmd_to_images/tmp.pdf',17)
    

/usr/local/miniconda3/envs/test-ocr/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_ca

⚠️  已为root用户设置XDG_RUNTIME_DIR：/tmp/runtime-root
✅ 初始化图像输出目录：mmd_to_images
✅ 主输出目录已准备：/workspace/DeepSeek-OCR/image_output
--------------------------------------------------

📌 处理第1/6个输入子目录：3276a5b991c49cf5f9a4af0f7d6fce67
  ✅ 输出子目录已创建：/workspace/DeepSeek-OCR/image_output/3276a5b991c49cf5f9a4af0f7d6fce67
images_input_path: temp/3276a5b991c49cf5f9a4af0f7d6fce67
output_dir: image_output/3276a5b991c49cf5f9a4af0f7d6fce67
page_num: 16


The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
<|ref|>title<|/ref|><|det|>[[306, 94, 680, 163]]<|/det|>
UNITED STATES DISTRICT COURT EASTERN DISTRICT OF MICHIGAN SOUTHERN DIVISION  

<|ref|>text<|/ref|><|det|>[[111, 188, 352, 210]]<|/det|>
United States of America,  

<|ref|>text<|/ref|><|det|>[[579, 212, 719, 234]]<|/det|>
Civil Case No:  

<|ref|>text<|/ref|><|det|>[[288, 237, 371, 257]]<|/det|>
Plaintiff,  

<|ref|>text<|/ref|><|det|>[[111, 262, 141, 280]]<|/det|>
vs.  

<|ref|>text<|/ref|><|det|>[[578, 260, 680, 281]]<|/det|>
Honorable  

<|ref|>text<|/ref|><|det|>[[111, 305, 456, 397]]<|/det|>
One (1) Mont Blanc Limited Edition. America's Signature for Freedom, Abraham Lincoln Themed Pen, Serial Number 11,  

<|ref|>text<|/ref|><|det|>[[287, 422, 456, 443]]<|/det|>
Defendant in rem.  

<|ref|>sub_title<|/ref|><|det|>[[367, 481, 622, 504]]<|/det|>
## Complaint For Forfeiture  

<|ref|>text<|/ref|><|det|>[[111, 515, 864, 621]]<|/det|>
Plaintiff, the United St

image: 0it [00:00, ?it/s]
other: 100%|██████████| 13/13 [00:00<00:00, 172550.48it/s]

...

image: 0it [00:00, ?it/s]
other: 100%|██████████| 55/55 [00:00<00:00, 251841.40it/s]

✅ markdown生成完成！
0



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 100, 1280])
<|ref|>text<|/ref|><|det|>[[117, 95, 205, 174]]<|/det|>
Zip Code Country Telephone Ext.  

<|ref|>text<|/ref|><|det|>[[448, 87, 595, 179]]<|/det|>
Redacted by NV DOIProvince County Redacted by NV DOI E- mail Address  

<|ref|>text<|/ref|><|det|>[[790, 117, 881, 145]]<|/det|>
Redacted by NV DOI  

<|ref|>sub_title<|/ref|><|det|>[[118, 185, 325, 205]]<|/det|>
## Complaint Against  

<|ref|>text<|/ref|><|det|>[[117, 209, 325, 288]]<|/det|>
I am complaining against My Insurance Company Agency Other  

<|ref|>text<|/ref|><|det|>[[460, 230, 875, 288]]<|/det|>
No Agent No Yes Other Party's Insurance Company No No  

<|ref|>sub_title<|/ref|><|det|>[[118, 300, 293, 317]]<|/det|>
## Agency Information  

<|ref|>text<|/ref|><|det|>[[116, 320, 238, 567]]<|/det|>
Agent Prefix Agent Middle Agent Suffix Agency Name Street/Apt# City State Zip Code Country Telephone Ext E- mail Address  

<|ref|>text<|/ref|><|det|>[[460, 320, 562, 3

KeyboardInterrupt: 